In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'test'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

%cd /content/drive/MyDrive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/test


# 2011~2022 데이터 통합

In [ ]:
# import glob

# path = '/content/drive/MyDrive/신한AI/데이터/경기도/통합본'
# file_name = '경기도_통합' # 통합한 csv 파일명

# file_list = glob.glob(path+'/*.csv')
# data = []
# result = pd.DataFrame()
# for files in file_list:    
#     df = pd.read_csv(files,encoding='cp949',error_bad_lines=False, warn_bad_lines=False)
#     data.append(df)
#     result = pd.concat(data, axis=0)
# result.to_csv(path+'/'+file_name+'.csv', encoding='cp949',index=False)


# 위 과정 반복

In [ ]:
# 시각화 위해 한글폰트 설치

# !pip install koreanize-matplotlib

# import koreanize_matplotlib

# %config InlineBackend.figure_format = 'retina'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#라이브러리, 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json

In [ ]:
gg_apt = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_아파트_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
gg_op = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_오피스텔_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
gg_multi = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_다세대_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
seoul_apt = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/서울/통합본/서울_아파트_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
seoul_op = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/서울/통합본/서울_오피스텔_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
seoul_multi = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/서울/통합본/서울_다세대_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
rate = pd.read_csv('/content/drive/MyDrive/신한AI/서울/한국은행 기준금리 및 여수신금리_27112307.csv')
kospi = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/kospi/코스피지수 과거 데이터 (1).csv', encoding = 'utf-8')

<ipython-input-7-d901905b2df6>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  gg_apt = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_아파트_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
<ipython-input-7-d901905b2df6>:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  gg_apt = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_아파트_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)
<ipython-input-7-d901905b2df6>:1: DtypeWarning: Columns (2,10,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  gg_apt = pd.read_csv('/content/drive/MyDrive/신한AI/데이터/경기도/통합본/경기도_아파트_통합.csv', encoding = 'cp949', error_bad_lines=False, warn_bad_lines=False)

## rate 데이터 처리

In [ ]:
rate.head(n=5)
rate = rate.drop(['통계표','단위','변환'], axis=1)
rate = rate.rename(columns={'계정항목':'날짜'}).transpose()
rate.reset_index(inplace=True)
rate.columns = ['날짜','한국은행기준금리']
rate = rate.drop(rate.index[0])
rate['날짜'] = rate['날짜'].str.replace('/','').astype('int64')

In [ ]:
rate.head()

,날짜,한국은행기준금리
1,202303,3.5
2,202302,3.5
3,202301,3.5
4,202212,3.25
5,202211,3.25


# KOSPI 데이터 처리

In [ ]:
kospi['날짜'] = kospi['날짜'].str.replace('-','')
kospi['날짜'] = kospi['날짜'].str.replace(' ','')
kospi['날짜'] = kospi['날짜'].str[:6].astype('int64')
# kospi['날짜'] = kospi['날짜'].astype('int64')

In [ ]:
kospi.head()

,날짜,종가,시가,고가,저가,거래량,변동 %
0,202301,"2,425.08","2,249.95","2,497.40","2,180.67",9.42B,8.44%
1,202212,"2,236.40","2,501.43","2,501.43","2,236.38",8.44B,-9.55%
2,202211,"2,472.53","2,301.58","2,499.43","2,296.44",12.75B,7.80%
3,202210,"2,293.61","2,189.51","2,295.51","2,162.87",11.77B,6.41%
4,202209,"2,155.49","2,443.00","2,453.95","2,134.77",9.38B,-12.81%


#서울 데이터 병합(오피스텔, 아파트, 다세대)

In [ ]:
# seoul_apt.info()
# seoul_multi.info()
# seoul_op.info()

In [ ]:
seoul_apt['구분'] = '아파트'
seoul_op['구분'] = '오피스텔'
seoul_multi['구분'] = '연립다세대'

In [ ]:
# seoul_ = pd.concat([seoul_apt, seoul_op, seoul_multi, seoul_single])
seoul_ = pd.concat([seoul_apt, seoul_op, seoul_multi])

In [ ]:
seoul_.sample(n=5)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),...,층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),구분,건물명
1360913,서울특별시 강서구 화곡동,1161,1161.0,0.0,강서동도센트리움,월세,14.6770,201602,4,"1,000",...,3.0,2012.0,공항대로46길 28,-,-,-,NaN,NaN,아파트,NaN
1768498,서울특별시 마포구 아현동,777,777.0,0.0,마포래미안푸르지오4단지,월세,84.3884,202212,6,0,...,19.0,2014.0,마포대로 195,202212~202412,갱신,사용,0,220.0,아파트,NaN
817157,서울특별시 중랑구 신내동,831,831.0,0.0,우디안4단지,월세,31.8500,202011,16,"6,670",...,4.0,2018.0,신내역로 160,-,-,-,NaN,NaN,아파트,NaN
1986471,서울특별시 동대문구 장안동,329-3,329.0,3.0,래미안장안2차,월세,81.0500,202102,6,"7,000",...,9.0,2007.0,장안벚꽃로 167,-,-,-,NaN,NaN,아파트,NaN
750481,서울특별시 성동구 옥수동,528-1,528.0,1.0,옥수파크힐스117동~125동,전세,59.5900,202007,11,"75,000",...,13.0,2016.0,매봉길 50,-,-,-,NaN,NaN,아파트,NaN


# 경기도 데이터 병합(오피스텔, 아파트, 다세대)

In [ ]:
# gg_apt.info()
# gg_op.info()
# gg_multi.info()

In [ ]:
gg_apt['구분'] = '아파트'
gg_op['구분'] = '오피스텔'
gg_multi['구분'] = '연립다세대'

In [ ]:
# gg_ = pd.concat([gg_apt, gg_op, gg_multi, gg_single])
gg_ = pd.concat([gg_apt, gg_op, gg_multi])

In [ ]:
gg_.sample(n=5)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),...,층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),구분,건물명
290266,경기도 이천시 부발읍 아미리,729-2,729,2,지평더웰아이,월세,30.250,201901,21,"2,000",...,5.0,2018.0,경충대로2092번길 25,-,-,-,NaN,NaN,오피스텔,NaN
52660,경기도 남양주시 퇴계원읍 퇴계원리,232,232,0,성원,전세,84.960,201708,28,"24,000",...,12.0,1999.0,도제원로 22,-,-,-,NaN,NaN,아파트,NaN
2649618,경기도 안성시 대덕면 진현리,555,555,0,유안,전세,59.080,201406,23,"6,000",...,6.0,2005.0,안성맞춤대로 1654,-,-,-,NaN,NaN,아파트,NaN
407730,경기도 의왕시 이동,218-65,218,65,NaN,전세,34.020,201903,2,"14,000",...,4.0,2018.0,새터말아랫로 8,-,-,-,NaN,NaN,연립다세대,정성빌딩
1449119,경기도 용인수지구 풍덕천동,1018,1018,0,신정마을임광,월세,101.647,202009,2,"35,000",...,2.0,2000.0,정평로 73,-,-,-,NaN,NaN,아파트,NaN


# 서울, 경기도 데이터 rate와 합치기

In [ ]:
seoul_ = pd.merge(seoul_, rate, left_on='계약년월', right_on='날짜').drop(['날짜'],axis=1)
gg_ = pd.merge(gg_, rate, left_on="계약년월", right_on='날짜').drop(['날짜'], axis=1)

#서울, 경기도 데이터 kospi와 합치기

In [ ]:
seoul_ = pd.merge(seoul_, kospi, left_on='계약년월', right_on='날짜').drop(['날짜'],axis=1)
gg_ = pd.merge(gg_, kospi, left_on='계약년월', right_on='날짜').drop(['날짜'],axis=1)

# 서울, 경기도 데이터 처리

In [ ]:
seoul_['보증금(만원)'] = seoul_['보증금(만원)'].astype('str')
seoul_['월세(만원)'] = seoul_['월세(만원)'].astype('str')
seoul_['보증금(만원)'] = seoul_['보증금(만원)'].str.replace(',','').astype('int64')
seoul_['월세(만원)'] = seoul_['월세(만원)'].str.replace(',','').astype('int64')
seoul_['계약년월'] = seoul_['계약년월'].astype('str')
seoul_['년도'] = seoul_['계약년월'].str[0:4].astype('int64')
seoul_['월'] = seoul_['계약년월'].str[4:].astype('int64')
seoul_ = seoul_.drop(['계약년월'],axis=1)

In [ ]:
gg_['보증금(만원)'] = gg_['보증금(만원)'].astype('str')
gg_['월세(만원)'] = gg_['월세(만원)'].astype('str')
gg_['보증금(만원)'] = gg_['보증금(만원)'].str.replace(',','').astype('int64')
gg_['월세(만원)'] = gg_['월세(만원)'].str.replace(',','').astype('int64')
gg_['계약년월'] = gg_['계약년월'].astype('str')
gg_['년도'] = gg_['계약년월'].str[0:4].astype('int64')
gg_['월'] = gg_['계약년월'].str[4:].astype('int64')
gg_ = gg_.drop(['계약년월'],axis=1)

In [ ]:
seoul_.sample(n=5)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약일,보증금(만원),월세(만원),...,건물명,한국은행기준금리,종가,시가,고가,저가,거래량,변동 %,년도,월
944702,서울특별시 도봉구 쌍문동,281-19,281.0,19.0,NaN,월세,69.44,23,6000,40,...,정한파크빌,3.0,"2,142.47","2,203.09","2,229.09","2,030.68",6.22B,-2.28%,2011,5
2432735,서울특별시 송파구 오금동,Sep-74,74.0,9.0,NaN,전세,42.22,30,13000,0,...,가락파크빌라(74-9),1.5,"1,912.06","1,954.47","1,954.52","1,830.06",7.30B,-2.51%,2016,1
1660510,서울특별시 영등포구 당산동,121-43,121.0,43.0,리스빌,월세,13.20,16,1000,45,...,NaN,1.5,"2,063.05","1,961.19","2,101.63","1,958.59",8.62B,4.84%,2019,9
2687088,서울특별시 구로구 신도림동,276-32,276.0,32.0,골든애비뉴,월세,19.81,25,3000,38,...,NaN,1.5,"1,991.97","2,033.15","2,056.12","1,942.85",9.00B,-1.85%,2015,11
3253935,서울특별시 성동구 금호동4가,800,800.0,0.0,대우,전세,59.97,24,46700,0,...,NaN,0.5,"3,012.95","2,976.81","3,180.94","2,947.24",30.03B,1.23%,2021,2


In [ ]:
gg_.sample(n=5)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약일,보증금(만원),월세(만원),...,건물명,한국은행기준금리,종가,시가,고가,저가,거래량,변동 %,년도,월
2389416,경기도 성남수정구 태평동,Mar-97,7297,3,NaN,전세,36.64,14,8000,0,...,삼환빌라,1.75,"2,195.44","2,211.93","2,241.76","2,167.36",6.88B,-0.43%,2019,2
1835350,경기도 수원팔달구 인계동,1118-8,1118,8,라비채빌딩,월세,29.69,18,500,55,...,NaN,0.5,"2,326.17","2,251.65","2,458.17","2,237.04",17.64B,3.41%,2020,8
2023412,경기도 화성시 향남읍 상신리,1331,1331,0,언덕마을18단지,월세,46.81,3,7006,11,...,NaN,0.5,"2,873.47","2,613.42","2,878.21","2,611.67",24.56B,10.89%,2020,12
1335839,경기도 안양동안구 비산동,576-7,576,7,비산힐스테이트,전세,82.20,26,35000,0,...,NaN,0.5,"3,296.68","3,207.12","3,316.08","3,199.18",35.35B,2.90%,2021,6
3155660,경기도 고양일산동구 중산동,1801,1801,0,하늘마을3단지(임대),월세,59.57,29,3190,27,...,NaN,3.25,"1,769.65","1,887.21","1,928.40","1,644.11",7.69B,-5.88%,2011,9


In [ ]:
# seoul_.keys()

In [ ]:
# gg_.keys()

In [ ]:
splited = seoul_['시군구'].str.split(' ', expand=True)
splited.columns = ['시/도','시/군/구','동']
seoul_=pd.concat([seoul_,splited], axis=1)
s_col1=seoul_.columns[-3:].to_list()
s_col2=seoul_.columns[:-3].to_list()
seoul_=seoul_[s_col1+s_col2]
seoul_ = seoul_.drop(['시군구'],axis=1)

In [ ]:
seoul_.head()

,시/도,시/군/구,동,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약일,...,건물명,한국은행기준금리,종가,시가,고가,저가,거래량,변동 %,년도,월
0,서울특별시,강남구,개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,21,...,NaN,2.5,"1,941.15","2,013.11","2,013.89","1,899.76",5.09B,-3.49%,2014,1
1,서울특별시,강남구,개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,4,...,NaN,2.5,"1,941.15","2,013.11","2,013.89","1,899.76",5.09B,-3.49%,2014,1
2,서울특별시,강남구,개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,8,...,NaN,2.5,"1,941.15","2,013.11","2,013.89","1,899.76",5.09B,-3.49%,2014,1
3,서울특별시,강남구,개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,전세,79.97,11,...,NaN,2.5,"1,941.15","2,013.11","2,013.89","1,899.76",5.09B,-3.49%,2014,1
4,서울특별시,강남구,개포동,652,652.0,0.0,개포우성3차,월세,104.43,17,...,NaN,2.5,"1,941.15","2,013.11","2,013.89","1,899.76",5.09B,-3.49%,2014,1


In [ ]:
splited_2 = gg_['시군구'].str.split(' ', expand=True)
splited_2.columns = ['시/도','시/군/구','동','리']
gg_=pd.concat([gg_,splited_2], axis=1)
col1=gg_.columns[-3:].to_list()
col2=gg_.columns[:-3].to_list()
gg_=gg_[col1+col2]
gg_ = gg_.drop(['시군구','리'],axis=1)

In [ ]:
gg_.head()

,시/군/구,동,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약일,보증금(만원),...,한국은행기준금리,종가,시가,고가,저가,거래량,변동 %,년도,월,시/도
0,가평군,가평읍,281,281,0,세대넥스빌,전세,122.280,5,6000,...,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",8.42B,2.03%,2017,1,경기도
1,가평군,가평읍,402-1,402,1,썬힐,월세,64.850,20,1000,...,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",8.42B,2.03%,2017,1,경기도
2,가평군,가평읍,771-1,771,1,에이원파란채,전세,84.987,25,10000,...,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",8.42B,2.03%,2017,1,경기도
3,가평군,가평읍,270,270,0,주공,월세,39.510,17,920,...,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",8.42B,2.03%,2017,1,경기도
4,가평군,가평읍,234,234,0,휴먼시아,월세,41.050,18,1475,...,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",8.42B,2.03%,2017,1,경기도


#seoul_new와 gg_new라는 변수에 복사해놓음

In [ ]:
seoul_new = seoul_
gg_new = gg_

In [ ]:
seoul_new = seoul_new.drop(['종전계약 보증금 (만원)', '종전계약 월세 (만원)', '건물명', '계약일','도로명', '단지명', '시/도','월','계약기간','계약구분','갱신요구권 사용','시가', '고가', '저가', '거래량', '변동 %'],axis=1)
gg_new = gg_new.drop(['종전계약 보증금 (만원)', '종전계약 월세 (만원)','건물명', '계약일','도로명', '단지명', '시/도','월','계약기간','계약구분','갱신요구권 사용','시가', '고가', '저가', '거래량', '변동 %'],axis=1)

In [ ]:
seoul_new.isnull().sum()

시/군/구           0
동               0
번지           2652
본번            235
부번            235
전월세구분           0
전용면적(㎡)        36
보증금(만원)         0
월세(만원)          0
층           18041
구분              0
한국은행기준금리        0
종가              0
년도              0
dtype: int64

In [ ]:
gg_new.isnull().sum()

시/군/구           0
동               0
번지           4553
본번              0
부번              0
전월세구분           0
전용면적(㎡)         5
보증금(만원)         0
월세(만원)          0
층           10541
구분              0
한국은행기준금리        0
종가              0
년도              0
dtype: int64

In [ ]:
seoul_new['년도'] = seoul_new['년도'].astype('float')
seoul_new['건물나이'] = seoul_new['년도'] - seoul_new['건축년도']
gg_new['년도'] = gg_new['년도'].astype('float')
gg_new['건물나이'] = gg_new['년도'] - gg_new['건축년도']
seoul_new = seoul_new.drop(['건축년도'], axis=1)
gg_new = gg_new.drop(['건축년도'], axis=1)

In [ ]:
# gg_2['보증금(만원)'] = pd.to_numeric(gg_2['보증금(만원)'], errors='coerce')
# gg_2['월세(만원)'] = pd.to_numeric(gg_2['월세(만원)'], errors='coerce')
# seoul_2['보증금(만원)'] = pd.to_numeric(seoul_2['보증금(만원)'], errors='coerce')
# seoul_2['월세(만원)'] = pd.to_numeric(seoul_2['월세(만원)'], errors='coerce')

In [ ]:
seoul_new.head()

,시/군/구,동,번지,본번,부번,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이
0,강남구,개포동,655-2,655.0,2.0,전세,77.75,33000,0,2.0,아파트,2.5,"1,941.15",2014.0,26.0
1,강남구,개포동,658-1,658.0,1.0,전세,79.97,34000,0,2.0,아파트,2.5,"1,941.15",2014.0,27.0
2,강남구,개포동,658-1,658.0,1.0,전세,79.97,28000,0,1.0,아파트,2.5,"1,941.15",2014.0,27.0
3,강남구,개포동,658-1,658.0,1.0,전세,79.97,35000,0,2.0,아파트,2.5,"1,941.15",2014.0,27.0
4,강남구,개포동,652,652.0,0.0,월세,104.43,40000,40,8.0,아파트,2.5,"1,941.15",2014.0,30.0


In [ ]:
gg_new.head()

,시/군/구,동,번지,본번,부번,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이
0,가평군,가평읍,281,281,0,전세,122.280,6000,0,1.0,아파트,1.25,"2,067.57",2017.0,21.0
1,가평군,가평읍,402-1,402,1,월세,64.850,1000,60,9.0,아파트,1.25,"2,067.57",2017.0,9.0
2,가평군,가평읍,771-1,771,1,전세,84.987,10000,0,10.0,아파트,1.25,"2,067.57",2017.0,10.0
3,가평군,가평읍,270,270,0,월세,39.510,920,16,6.0,아파트,1.25,"2,067.57",2017.0,18.0
4,가평군,가평읍,234,234,0,월세,41.050,1475,8,3.0,아파트,1.25,"2,067.57",2017.0,7.0


In [ ]:
# null 삭제
seoul_dropna = seoul_new.dropna()
gg_dropna = gg_new.dropna()

In [ ]:
seoul_dropna.to_csv('./서울_한글.csv',index=False)
gg_dropna.to_csv('./경기도_한글.csv',index=False)

In [ ]:
seoul_df = seoul_dropna
gg_df = gg_dropna

In [ ]:
#전세는 0, 월세 1
seoul_df.loc[seoul_df['전월세구분']=='전세','전월세구분'] = 0
seoul_df.loc[seoul_df['전월세구분']=='월세','전월세구분'] = 1
gg_df.loc[gg_df['전월세구분']=='전세','전월세구분'] = 0
gg_df.loc[gg_df['전월세구분']=='월세','전월세구분'] = 1

In [ ]:
#아파트 0, 오피스텔 1, 연립다세대 2
seoul_df.loc[seoul_df['구분']=='아파트','구분'] = 0
seoul_df.loc[seoul_df['구분']=='오피스텔','구분'] = 1
seoul_df.loc[seoul_df['구분']=='연립다세대','구분'] = 2
gg_df.loc[gg_df['구분']=='아파트','구분'] = 0
gg_df.loc[gg_df['구분']=='오피스텔','구분'] = 1
gg_df.loc[gg_df['구분']=='연립다세대','구분'] = 2

In [ ]:
seoul_df = seoul_df.drop(['번지','본번','부번'], axis=1)
gg_df = gg_df.drop(['번지','본번','부번'], axis=1)

In [ ]:
gg_df

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이
0,가평군,가평읍,0,122.280,6000,0,1.0,0,1.25,"2,067.57",2017.0,21.0
1,가평군,가평읍,1,64.850,1000,60,9.0,0,1.25,"2,067.57",2017.0,9.0
2,가평군,가평읍,0,84.987,10000,0,10.0,0,1.25,"2,067.57",2017.0,10.0
3,가평군,가평읍,1,39.510,920,16,6.0,0,1.25,"2,067.57",2017.0,18.0
4,가평군,가평읍,1,41.050,1475,8,3.0,0,1.25,"2,067.57",2017.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3665275,화성시,향남읍,0,47.100,5000,0,2.0,2,2.25,"2,020.09",2014.0,22.0
3665276,화성시,향남읍,0,56.280,5500,0,4.0,2,2.25,"2,020.09",2014.0,22.0
3665277,화성시,향남읍,0,39.260,4000,0,1.0,2,2.25,"2,020.09",2014.0,23.0
3665278,화성시,향남읍,0,75.870,6000,0,1.0,2,2.25,"2,020.09",2014.0,17.0


In [ ]:
gg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3636719 entries, 0 to 3665279
Data columns (total 12 columns):
 #   Column    Dtype  
---  ------    -----  
 0   시/군/구     object 
 1   동         object 
 2   전월세구분     object 
 3   전용면적(㎡)   float64
 4   보증금(만원)   int64  
 5   월세(만원)    int64  
 6   층         float64
 7   구분        object 
 8   한국은행기준금리  object 
 9   종가        object 
 10  년도        float64
 11  건물나이      float64
dtypes: float64(4), int64(2), object(6)
memory usage: 360.7+ MB


In [ ]:
# 서울 시/군/구 Label Encoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(seoul_df['시/군/구'])

seoul_df["시군구_Encoder"] = encoder.transform(seoul_df['시/군/구'])
seoul_df

['강남구' '강동구' '강북구' '강서구' '관악구' '광진구' '구로구' '금천구' '노원구' '도봉구' '동대문구' '동작구'
 '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구' '영등포구' '용산구' '은평구' '종로구' '중구'
 '중랑구']



,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder
0,강남구,개포동,0,77.75,33000,0,2.0,0,2.5,"1,941.15",2014.0,26.0,0
1,강남구,개포동,0,79.97,34000,0,2.0,0,2.5,"1,941.15",2014.0,27.0,0
2,강남구,개포동,0,79.97,28000,0,1.0,0,2.5,"1,941.15",2014.0,27.0,0
3,강남구,개포동,0,79.97,35000,0,2.0,0,2.5,"1,941.15",2014.0,27.0,0
4,강남구,개포동,1,104.43,40000,40,8.0,0,2.5,"1,941.15",2014.0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622986,중랑구,중화동,0,29.73,18350,0,2.0,2,1.0,"2,977.65",2021.0,6.0,24
3622987,중랑구,중화동,0,28.49,18000,0,4.0,2,1.0,"2,977.65",2021.0,6.0,24
3622988,중랑구,중화동,1,22.40,1000,63,2.0,2,1.0,"2,977.65",2021.0,4.0,24
3622989,중랑구,중화동,0,83.61,32500,0,2.0,2,1.0,"2,977.65",2021.0,18.0,24


In [ ]:
# 서울_동 label encoding

encoder.fit(seoul_df['동'])

seoul_df["동_Encoder"] = encoder.transform(seoul_df['동'])
seoul_df

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder,동_Encoder
0,강남구,개포동,0,77.75,33000,0,2.0,0,2.5,"1,941.15",2014.0,26.0,0,9
1,강남구,개포동,0,79.97,34000,0,2.0,0,2.5,"1,941.15",2014.0,27.0,0,9
2,강남구,개포동,0,79.97,28000,0,1.0,0,2.5,"1,941.15",2014.0,27.0,0,9
3,강남구,개포동,0,79.97,35000,0,2.0,0,2.5,"1,941.15",2014.0,27.0,0,9
4,강남구,개포동,1,104.43,40000,40,8.0,0,2.5,"1,941.15",2014.0,30.0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622986,중랑구,중화동,0,29.73,18350,0,2.0,2,1.0,"2,977.65",2021.0,6.0,24,326
3622987,중랑구,중화동,0,28.49,18000,0,4.0,2,1.0,"2,977.65",2021.0,6.0,24,326
3622988,중랑구,중화동,1,22.40,1000,63,2.0,2,1.0,"2,977.65",2021.0,4.0,24,326
3622989,중랑구,중화동,0,83.61,32500,0,2.0,2,1.0,"2,977.65",2021.0,18.0,24,326


In [ ]:
# 경기 시/군/구 label encoding
encoder.fit(gg_df['시/군/구'])

gg_df["시군구_Encoder"] = encoder.transform(gg_df['시/군/구'])
gg_df

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder
0,가평군,가평읍,0,122.280,6000,0,1.0,0,1.25,"2,067.57",2017.0,21.0,0
1,가평군,가평읍,1,64.850,1000,60,9.0,0,1.25,"2,067.57",2017.0,9.0,0
2,가평군,가평읍,0,84.987,10000,0,10.0,0,1.25,"2,067.57",2017.0,10.0,0
3,가평군,가평읍,1,39.510,920,16,6.0,0,1.25,"2,067.57",2017.0,18.0,0
4,가평군,가평읍,1,41.050,1475,8,3.0,0,1.25,"2,067.57",2017.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3665275,화성시,향남읍,0,47.100,5000,0,2.0,2,2.25,"2,020.09",2014.0,22.0,41
3665276,화성시,향남읍,0,56.280,5500,0,4.0,2,2.25,"2,020.09",2014.0,22.0,41
3665277,화성시,향남읍,0,39.260,4000,0,1.0,2,2.25,"2,020.09",2014.0,23.0,41
3665278,화성시,향남읍,0,75.870,6000,0,1.0,2,2.25,"2,020.09",2014.0,17.0,41


In [ ]:
# 경기_동 label encoding

encoder.fit(gg_df['동'])

gg_df["동_Encoder"] = encoder.transform(gg_df['동'])
gg_df

,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이,시군구_Encoder,동_Encoder
0,가평군,가평읍,0,122.280,6000,0,1.0,0,1.25,"2,067.57",2017.0,21.0,0,8
1,가평군,가평읍,1,64.850,1000,60,9.0,0,1.25,"2,067.57",2017.0,9.0,0,8
2,가평군,가평읍,0,84.987,10000,0,10.0,0,1.25,"2,067.57",2017.0,10.0,0,8
3,가평군,가평읍,1,39.510,920,16,6.0,0,1.25,"2,067.57",2017.0,18.0,0,8
4,가평군,가평읍,1,41.050,1475,8,3.0,0,1.25,"2,067.57",2017.0,7.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3665275,화성시,향남읍,0,47.100,5000,0,2.0,2,2.25,"2,020.09",2014.0,22.0,41,597
3665276,화성시,향남읍,0,56.280,5500,0,4.0,2,2.25,"2,020.09",2014.0,22.0,41,597
3665277,화성시,향남읍,0,39.260,4000,0,1.0,2,2.25,"2,020.09",2014.0,23.0,41,597
3665278,화성시,향남읍,0,75.870,6000,0,1.0,2,2.25,"2,020.09",2014.0,17.0,41,597


In [ ]:
seoul_df.to_csv('./서울_인코딩_임시.csv',index=False)
gg_df.to_csv('./경기도_인코딩_임시.csv',index=False)

In [ ]:
seoul_n

,시군구_Encoder,동_Encoder,시/군/구,동,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,구분,한국은행기준금리,종가,년도,건물나이
0,0,9,강남구,개포동,0,77.75,33000,0,2.0,0,2.5,"1,941.15",2014.0,26.0
1,0,9,강남구,개포동,0,79.97,34000,0,2.0,0,2.5,"1,941.15",2014.0,27.0
2,0,9,강남구,개포동,0,79.97,28000,0,1.0,0,2.5,"1,941.15",2014.0,27.0
3,0,9,강남구,개포동,0,79.97,35000,0,2.0,0,2.5,"1,941.15",2014.0,27.0
4,0,9,강남구,개포동,1,104.43,40000,40,8.0,0,2.5,"1,941.15",2014.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622986,24,326,중랑구,중화동,0,29.73,18350,0,2.0,2,1.0,"2,977.65",2021.0,6.0
3622987,24,326,중랑구,중화동,0,28.49,18000,0,4.0,2,1.0,"2,977.65",2021.0,6.0
3622988,24,326,중랑구,중화동,1,22.40,1000,63,2.0,2,1.0,"2,977.65",2021.0,4.0
3622989,24,326,중랑구,중화동,0,83.61,32500,0,2.0,2,1.0,"2,977.65",2021.0,18.0


In [ ]:
s_coll=seoul_df.columns[-2:].to_list()
s_coll2=seoul_df.columns[:-2].to_list()
seoul_n=seoul_df[s_coll+s_coll2]
seoul_n = seoul_n.drop(['시/군/구','동'],axis=1)

In [ ]:
s_coll=seoul_df.columns[-2:].to_list()
s_coll2=seoul_df.columns[:-2].to_list()
seoul_n=seoul_df[s_coll+s_coll2]
seoul_n = seoul_n.drop(['시/군/구','동'],axis=1)

In [ ]:
# pd.concat([seoul_dropna.drop(['시/군/구','구분','동'],axis=1),pd.get_dummies(seoul_dropna['시/군/구']), pd.get_dummies(seoul_dropna['구분'])],axis=1)

In [ ]:
# 한글파일

In [ ]:
# seoul_df = pd.concat([seoul_dropna.drop(['시/군/구','구분','동'],axis=1)
# gg_df = pd.concat([gg_dropna.drop(['시/군/구','구분','동'],axis=1)

1

In [ ]:
# 일단 시군구만 사용함
# seoul_df = pd.concat([seoul_dropna.drop(['시/군/구','구분','동'],axis=1),pd.get_dummies(seoul_dropna['시/군/구']), pd.get_dummies(seoul_dropna['구분'])],axis=1)

In [ ]:
# 일단 시군구만 사용함
# gg_df = pd.concat([gg_dropna.drop(['시/군/구','구분','동'],axis=1),pd.get_dummies(gg_dropna['시/군/구']), pd.get_dummies(gg_dropna['구분'])],axis=1)

,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,한국은행기준금리,종가,시가,고가,저가,...,의정부시,이천시,파주시,평택시,포천시,하남시,화성시,아파트,연립다세대,오피스텔
3,1,39.5100,920.0,16,6.0,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",...,0,0,0,0,0,0,0,1,0,0
29,1,114.7500,200.0,20,12.0,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",...,0,0,0,0,0,0,0,1,0,0
54,1,45.7200,500.0,38,2.0,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",...,0,0,0,0,0,0,0,1,0,0
209,1,21.7500,539.0,19,13.0,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",...,0,0,0,0,0,0,0,1,0,0
347,1,45.3600,200.0,30,11.0,1.25,"2,067.57","2,022.23","2,091.02","2,015.68",...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3665258,1,17.8200,500.0,41,3.0,2.25,"2,020.09","2,067.80","2,072.59","2,007.30",...,0,0,0,0,0,0,1,0,1,0
3665261,1,17.7300,500.0,58,4.0,2.25,"2,020.09","2,067.80","2,072.59","2,007.30",...,0,0,0,0,0,0,1,0,1,0
3665267,1,33.8900,300.0,30,3.0,2.25,"2,020.09","2,067.80","2,072.59","2,007.30",...,0,0,0,0,0,0,1,0,1,0
3665272,1,21.5307,300.0,40,3.0,2.25,"2,020.09","2,067.80","2,072.59","2,007.30",...,0,0,0,0,0,0,1,0,1,0


In [ ]:
gg_df.columns
# 

Index(['전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '한국은행기준금리', '종가', '변동 %',
       '계약년도', '건물나이', '가평군', '고양덕양구', '고양일산동구', '고양일산서구', '과천시', '광명시', '광주시',
       '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남분당구', '성남수정구', '성남중원구',
       '수원권선구', '수원영통구', '수원장안구', '수원팔달구', '시흥시', '안산단원구', '안산상록구', '안성시',
       '안양동안구', '안양만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인기흥구', '용인수지구',
       '용인처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '아파트',
       '연립다세대', '오피스텔'],
      dtype='object')

In [ ]:
seoul_df.columns

Index(['전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '한국은행기준금리', '종가', '변동 %',
       '계약년도', '건물나이', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구', '아파트', '연립다세대',
       '오피스텔'],
      dtype='object')

In [ ]:
seoul_df.rename(columns={'년도':'계약년도'},inplace=True)

In [ ]:
gg_df.rename(columns={'년도':'계약년도'},inplace=True)

In [ ]:
gg_df

,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,한국은행기준금리,종가,변동 %,계약년도,건물나이,...,의정부시,이천시,파주시,평택시,포천시,하남시,화성시,아파트,연립다세대,오피스텔
3,1,39.5100,920.0,16,6.0,1.25,"2,067.57",2.03%,2017.0,18.0,...,0,0,0,0,0,0,0,1,0,0
29,1,114.7500,200.0,20,12.0,1.25,"2,067.57",2.03%,2017.0,13.0,...,0,0,0,0,0,0,0,1,0,0
54,1,45.7200,500.0,38,2.0,1.25,"2,067.57",2.03%,2017.0,34.0,...,0,0,0,0,0,0,0,1,0,0
209,1,21.7500,539.0,19,13.0,1.25,"2,067.57",2.03%,2017.0,1.0,...,0,0,0,0,0,0,0,1,0,0
347,1,45.3600,200.0,30,11.0,1.25,"2,067.57",2.03%,2017.0,23.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3665258,1,17.8200,500.0,41,3.0,2.25,"2,020.09",-2.34%,2014.0,1.0,...,0,0,0,0,0,0,1,0,1,0
3665261,1,17.7300,500.0,58,4.0,2.25,"2,020.09",-2.34%,2014.0,0.0,...,0,0,0,0,0,0,1,0,1,0
3665267,1,33.8900,300.0,30,3.0,2.25,"2,020.09",-2.34%,2014.0,18.0,...,0,0,0,0,0,0,1,0,1,0
3665272,1,21.5307,300.0,40,3.0,2.25,"2,020.09",-2.34%,2014.0,2.0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
seoul_df.corr(method="pearson")

In [ ]:
gg_df.corr(method="pearson")

In [ ]:
# seoul_year = seoul_[seoul_["전월세구분"]=="전세"]
# seoul_month = seoul_[seoul_["전월세구분"]=="월세"]
# gg_year = gg_[gg_["전월세구분"]=="전세"]
# gg_month = gg_[gg_["전월세구분"]=="월세"]

In [ ]:
# seoul_year.groupby('시/군/구')['보증금(만원)'].describe()

In [ ]:
# seoul_month.groupby('시/군/구')['보증금(만원)'].describe()

In [ ]:
# gg_month.groupby('시/군/구')['보증금(만원)'].describe()

In [ ]:
# gg_year.groupby('시/군/구')['보증금(만원)'].describe()

In [ ]:
# fig, (ax_1, ax_2) = plt.subplots(1,2, figsize=(20,10))
# ax_1 = sns.heatmap(seoul_year.corr(), annot=True, annot_kws=dict(color='black'), cmap='Blues', ax = ax_1)
# ax_2 = sns.heatmap(seoul_month.corr(), annot=True, annot_kws=dict(color='black'), cmap='Greens', ax = ax_2)
# plt.show()

In [ ]:
# fig, (ax_3, ax_4) = plt.subplots(1,2, figsize=(20,10))
# ax_3 = sns.heatmap(gg_year.corr(), annot=True, annot_kws=dict(color='black'), cmap='Reds', ax = ax_3)
# ax_4 = sns.heatmap(gg_month.corr(), annot=True, annot_kws=dict(color='black'), cmap='Purples', ax = ax_4)

In [ ]:
# # 서울 지역별 평균 전세

# plt.figure(figsize=(18, 4))
# sns.barplot(data=seoul_year, x="시/군/구", y="보증금(만원)", ci=None)

In [ ]:
# # 서울 지역별 평균 월세

# plt.figure(figsize=(18, 4))
# sns.barplot(data=seoul_month, x="시/군/구", y="월세(만원)", ci=None)

In [ ]:
# gg_year.groupby(by=["시/군/구"])["보증금(만원)"].mean().sort_values(ascending=False).plot(kind="bar", rot=0, title="경기 전세 평균 보증금", figsize=(30, 4))

In [ ]:
# plt.figure(figsize=(30, 4))
# sns.barplot(data=gg_year, x="시/군/구", y="보증금(만원)")

In [ ]:
# gg_month.groupby(by=["시/군/구"])["월세(만원)"].mean().sort_values(ascending=False).plot(kind="bar", rot=0, title="경기 평균 월세", figsize=(30, 4))

In [ ]:
# m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

# m.choropleth(
#     geo_data=state_geo,
#     data=seoul_year,
#     columns=['시/군/구', '월세(만원)'],
#     key_on='feature.properties.name',
#     fill_color='Blues',
#     fill_opacity=0.7,
#     line_opacity=0.3,
#     color = 'gray',
#     legend_name = '월세'
# )

In [ ]:
# seoul_year["평당전세가"] = seoul_year["보증금(만원)"]/(seoul_year["전용면적(㎡)"] / 3.3)
# seoul_month["평당월세"] = seoul_month["월세(만원)"]/(seoul_month["전용면적(㎡)"] / 3.3)
# gg_year["평당전세가"] = gg_year["보증금(만원)"]/(gg_year["전용면적(㎡)"] / 3.3)
# gg_month["평당월세"] = gg_month["월세(만원)"]/(gg_month["전용면적(㎡)"] / 3.3)

In [ ]:
# seoul_month.isnull().sum()

1. 버려야되는 feature 선택하기
2. 건축 년도와 같이 필요한 feature의 결측치 대체 진행하기
3. 이상치 찾기(boxplot, DBSCAN, t-SNE)
4. 문자열 데이터 수치형으로 바꾸기 (label-encoding, one-hot-incoding 등)
5. 머신러닝 알고리즘에 적용할 수 있도록 reshape 진행하기

In [ ]:
seoul_df.to_csv('./서울.csv',index=False)
gg_df.to_csv('./경기도.csv',index=False)

1. 동, 살리기
2. 한글 정리파일 vs 라벨링 정리파일
3. 원핫인코딩 지워 (아파트놈)
4. 